In [15]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: torch 2.7.0+cpu
Uninstalling torch-2.7.0+cpu:
  Successfully uninstalled torch-2.7.0+cpu
Found existing installation: torchvision 0.22.0+cpu
Uninstalling torchvision-0.22.0+cpu:
  Successfully uninstalled torchvision-0.22.0+cpu
Found existing installation: torchaudio 2.7.0+cpu
Uninstalling torchaudio-2.7.0+cpu:
  Successfully uninstalled torchaudio-2.7.0+cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (176.0 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (2.0 MB)
Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (1.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requir

In [2]:
# 1. Install required packages
!pip install transformers nltk textstat plotly umls-api

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 24.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.2 MB/s eta 0:00:00
  Created wheel for umls-api: filename=umls_api-0.1.0-py3-none-any.whl size=3382 sha256=92dcd44074ad47658f5e079b233ca3618243416612562099af57a26657ab2c5a
  Stored in directory: /root/.cache/pip/wheels/ca/92/1a/5526604053d0fc10179a96ec9b982760422143f0be9c63d9e8
Successfully built umls-api
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.2
    Uninstalling cachetools-5.5.2:
      Successfully uninstalled cachetools-5.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 2.28.1 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but yo

In [16]:
# %%
# 2. Imports and Setup
import re
import os
import torch
import textstat
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import plotly.express as px
import plotly.graph_objects as go

# UMLS client (stubbed if not installed)
try:
    from umls_api.umls import UMLSClient
    UMLS_AVAILABLE = True
except ImportError:
    UMLS_AVAILABLE = False

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# 3. UMLS Configuration 
if UMLS_AVAILABLE:
    UMLS_API_KEY = os.getenv('UMLS_API_KEY', '')
    umls_client = UMLSClient(api_key=UMLS_API_KEY)
    def get_umls_definition(term: str) -> str:
        try:
            results = umls_client.search(term, source='CHV')
            if results:
                return results[0]['definition']
            results = umls_client.search(term)
            if results:
                return results[0]['definition']
        except Exception:
            pass
        return None
else:
    def get_umls_definition(term: str) -> str:
        
        return None

In [18]:
# %%
# 4. Constants & Model Loading
MODEL_NAME = "mrm8488/t5-small-finetuned-text-simplification"
MAX_LENGTH = 512
MIN_LENGTH = 50
NO_REPEAT_NGRAM_SIZE = 3

print(f"Loading simplification model '{MODEL_NAME}'…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Loading simplification model 'mrm8488/t5-small-finetuned-text-simplification'…


In [19]:
# %%
# 5. Lay-Person Medical Dictionary
MEDICAL_DICTIONARY = {
    # Cardiovascular (50)
    "myocardial infarction":             "heart attack",
    "coronary artery disease":           "heart artery blockage",
    "congestive heart failure":          "heart cannot pump enough blood",
    "arrhythmia":                        "abnormal heartbeat",
    "atrial fibrillation":               "irregular heart rhythm",
    "bradycardia":                       "slow heart rate",
    "tachycardia":                       "fast heart rate",
    "angioplasty":                       "artery widening procedure",
    "stent":                             "small tube to keep the artery open",
    "aortic dissection":                 "tear in the main body artery",
    "atherosclerosis":                   "hardening of the arteries",
    "endocarditis":                      "inner heart lining infection",
    "pericarditis":                      "outer heart lining inflammation",
    "cardiomyopathy":                    "heart muscle disease",
    "cardiomegaly":                      "enlarged heart",
    "deep vein thrombosis":              "leg blood clot",
    "pulmonary embolism":                "lung blood clot",
    "varicose veins":                    "swollen leg veins",
    "hypertension":                      "high blood pressure",
    "hypotension":                       "low blood pressure",
    "hyperlipidemia":                    "high cholesterol",
    "hypercholesterolemia":              "high cholesterol",
    "hypertriglyceridemia":              "high blood fats",
    "cardiogenic shock":                 "severe heart failure shock",
    "vasculitis":                        "blood vessel inflammation",
    "phlebitis":                         "vein inflammation",
    "myocardial ischemia":               "low blood flow to the heart",
    "peripheral artery disease":         "blocked leg arteries",
    "endovascular surgery":              "vessel repair from inside",
    "valvular heart disease":            "heart valve problem",
    "mitral valve prolapse":             "leaky heart valve",
    "aortic stenosis":                   "narrowed heart valve",
    "cardiac tamponade":                 "fluid compressing the heart",
    "heart block":                       "electrical conduction problem",
    "ventricular fibrillation":          "life-threatening irregular heartbeat",
    "supraventricular tachycardia":      "fast heart rhythm from upper chambers",
    "heart murmur":                      "abnormal heart sound",
    "claudication":                      "leg pain from poor circulation",
    "Raynaud's phenomenon":              "cold-induced finger color changes",
    "thrombophlebitis":                  "vein inflammation with clot",
    "arteriovenous fistula":             "abnormal artery-vein connection",
    "cardiac arrest":                    "heart suddenly stops beating",
    "cor pulmonale":                     "right heart failure from lung disease",
    "hypertensive crisis":               "dangerously high blood pressure",
    "orthostatic hypotension":           "blood pressure drop when standing",
    "palpitations":                      "awareness of heartbeat",
    "syncope":                           "fainting",
    "premature ventricular contractions": "extra heartbeats",
    "cardiac catheterization":           "heart tube examination",
    "echocardiogram":                    "heart ultrasound",

    # Respiratory (40)
    "pneumonia":                         "lung infection",
    "bronchitis":                        "windpipe inflammation",
    "asthma":                            "airway narrowing disease",
    "chronic obstructive pulmonary disease": "long-term lung disease",
    "emphysema":                         "air sac damage in lungs",
    "pulmonary fibrosis":                "lung tissue scarring",
    "pleural effusion":                  "fluid around the lung",
    "pneumothorax":                      "collapsed lung",
    "respiratory acidosis":              "acid build-up due to poor breathing",
    "respiratory alkalosis":             "low CO₂ in blood due to over-breathing",
    "hypoxemia":                         "low blood oxygen",
    "hypercapnia":                       "high CO₂ in blood",
    "acute respiratory distress syndrome": "severe lung failure",
    "sleep apnea":                       "breathing stops during sleep",
    "pulmonary hypertension":            "high lung blood pressure",
    "laryngitis":                        "voice box inflammation",
    "pharyngitis":                       "throat inflammation",
    "tracheostomy":                      "voice box opening surgery",
    "intubation":                        "tube placed in airway",
    "bronchoscopy":                      "windpipe camera test",
    "bronchiectasis":                    "abnormal airway widening",
    "cystic fibrosis":                   "thick mucus lung disease",
    "pleurisy":                          "lung lining inflammation",
    "hemothorax":                        "blood in chest cavity",
    "empyema":                           "pus in chest cavity",
    "pulmonary edema":                   "fluid in lungs",
    "tuberculosis":                      "bacterial lung infection",
    "pertussis":                         "whooping cough",
    "influenza":                         "flu",
    "coronavirus":                       "COVID-19 virus",
    "SARS":                              "severe acute respiratory syndrome",
    "MERS":                              "Middle East respiratory syndrome",
    "rhinitis":                          "nose inflammation",
    "sinusitis":                         "sinus inflammation",
    "nasal polyps":                      "noncancerous nose growths",
    "epistaxis":                         "nosebleed",
    "aspiration pneumonia":              "lung infection from inhaling substances",
    "pulmonary nodule":                  "small lung growth",
    "mesothelioma":                      "lung lining cancer",
    "lung cancer":                       "malignant lung tumor",

    # Neurological (50)
    "stroke":                            "brain attack",
    "transient ischemic attack":        "mini stroke",
    "seizures":                          "uncontrolled brain activity",
    "epilepsy":                          "seizure disorder",
    "parkinson's disease":               "nerve disorder affecting movement",
    "alzheimer's disease":               "brain memory loss disease",
    "dementia":                          "memory and thinking decline",
    "multiple sclerosis":                "nerve insulation damage",
    "neuropathy":                        "nerve damage",
    "encephalopathy":                    "brain function disorder",
    "meningitis":                        "brain/spinal cord lining infection",
    "encephalitis":                      "brain inflammation",
    "intracranial hemorrhage":           "brain bleeding",
    "subarachnoid hemorrhage":           "bleeding around the brain",
    "subdural hematoma":                 "blood under brain covering",
    "traumatic brain injury":            "brain damage from trauma",
    "concussion":                        "mild brain injury",
    "hydrocephalus":                     "fluid build-up in brain",
    "cerebral palsy":                    "movement disorder from brain injury",
    "amyotrophic lateral sclerosis":     "progressive nerve disease",
    "bell's palsy":                      "facial nerve paralysis",
    "guillain-barre syndrome":           "immune system attacks nerves",
    "myasthenia gravis":                 "muscle weakness from nerve problem",
    "huntington's disease":              "inherited nerve degeneration",
    "tourette syndrome":                 "involuntary movements and sounds",
    "restless legs syndrome":            "uncomfortable leg sensations",
    "migraine":                          "severe headache",
    "cluster headache":                  "extremely painful headache",
    "tension headache":                  "stress-related headache",
    "trigeminal neuralgia":              "severe facial pain",
    "sciatica":                          "nerve pain down the leg",
    "radiculopathy":                     "pinched nerve",
    "carpal tunnel syndrome":            "wrist nerve compression",
    "peripheral neuropathy":             "nerve damage in extremities",
    "autonomic neuropathy":              "automatic nerve system damage",
    "neurofibromatosis":                 "nerve tumor disorder",
    "spina bifida":                      "spinal cord development defect",
    "syringomyelia":                     "cyst in spinal cord",
    "narcolepsy":                        "uncontrollable sleepiness",
    "insomnia":                          "difficulty sleeping",
    "vertigo":                           "spinning sensation",
    "meniere's disease":                 "inner ear balance disorder",
    "ataxia":                            "lack of muscle control",
    "dysarthria":                        "difficulty speaking",
    "aphasia":                           "language impairment",
    "agnosia":                           "inability to recognize objects",
    "apraxia":                           "inability to perform movements",
    "delirium":                          "acute confusion",
    "coma":                              "prolonged unconsciousness",
    "vegetative state":                  "wakeful unconsciousness",

    # Gastrointestinal (40)
    "pancreatitis":                      "pancreas inflammation",
    "hepatitis":                         "liver inflammation",
    "cirrhosis":                         "liver scarring",
    "colitis":                           "colon inflammation",
    "crohn's disease":                   "intestinal inflammation",
    "ulcerative colitis":                "colon ulcer inflammation",
    "gastritis":                         "stomach lining inflammation",
    "peptic ulcer disease":              "stomach/duodenal ulcer",
    "gastroesophageal reflux disease":   "acid reflux",
    "irritable bowel syndrome":          "bowel function disorder",
    "cholecystitis":                     "gallbladder inflammation",
    "cholelithiasis":                    "gallstones",
    "appendicitis":                      "appendix inflammation",
    "diverticulitis":                    "colon pouch inflammation",
    "ileus":                             "intestinal blockage",
    "hepatocellular carcinoma":          "liver cancer",
    "pancreatic cancer":                 "pancreas cancer",
    "ascites":                           "fluid in abdomen",
    "jaundice":                          "yellow skin",
    "hepatoblastoma":                    "child liver tumor",
    "esophageal varices":                "swollen esophagus veins",
    "gastric cancer":                    "stomach cancer",
    "colorectal cancer":                 "colon/rectum cancer",
    "celiac disease":                    "gluten intolerance",
    "lactose intolerance":               "inability to digest milk sugar",
    "malabsorption":                     "poor nutrient absorption",
    "whipple's disease":                 "rare digestive disorder",
    "hemochromatosis":                   "iron overload",
    "wilson's disease":                  "copper accumulation",
    "hiatal hernia":                     "stomach protrusion into chest",
    "achalasia":                         "esophagus movement disorder",
    "barrett's esophagus":               "esophagus lining changes",
    "dysphagia":                         "difficulty swallowing",
    "odynophagia":                       "painful swallowing",
    "hematemesis":                       "vomiting blood",
    "melena":                            "black, tarry stools",
    "steatorrhea":                       "fatty stools",
    "constipation":                      "difficulty passing stools",
    "diarrhea":                          "loose, watery stools",
    "hemorrhoids":                       "swollen rectal veins",

    # Renal (30)
    "nephritis":                         "kidney inflammation",
    "nephropathy":                       "kidney disease",
    "acute kidney injury":               "sudden kidney failure",
    "chronic kidney disease":            "long-term kidney failure",
    "glomerulonephritis":                "kidney filter inflammation",
    "nephrolithiasis":                   "kidney stones",
    "urinary tract infection":           "urine infection",
    "pyelonephritis":                    "kidney infection",
    "renal failure":                     "kidneys not working",
    "dialysis":                          "blood cleaning treatment",
    "renal artery stenosis":             "kidney artery narrowing",
    "hydronephrosis":                    "swollen kidney",
    "proteinuria":                       "protein in urine",
    "hematuria":                         "blood in urine",
    "glomerular filtration rate":        "kidney function test",
    "polycystic kidney disease":         "kidney cysts",
    "nephrotic syndrome":                "kidney leakage disorder",
    "nephritic syndrome":                "kidney inflammation disorder",
    "bladder cancer":                    "urinary bladder malignancy",
    "renal cell carcinoma":              "kidney cancer",
    "wilms tumor":                       "childhood kidney cancer",
    "urinary incontinence":              "loss of bladder control",
    "overactive bladder":                "frequent urination urge",
    "interstitial cystitis":             "chronic bladder inflammation",
    "urethritis":                        "urethra inflammation",
    "prostatitis":                       "prostate inflammation",
    "benign prostatic hyperplasia":      "enlarged prostate",
    "renal tubular acidosis":            "kidney acid regulation disorder",
    "fanconi syndrome":                  "kidney tubule disorder",
    "bladder stones":                    "urinary bladder calculi",

    # Endocrine (30)
    "diabetes mellitus":                 "high blood sugar disease",
    "type 1 diabetes":                   "autoimmune diabetes",
    "type 2 diabetes":                   "insulin resistance diabetes",
    "hyperthyroidism":                   "overactive thyroid",
    "hypothyroidism":                    "underactive thyroid",
    "goiter":                            "thyroid gland enlargement",
    "adrenal insufficiency":             "low adrenal hormone levels",
    "cushing's syndrome":                "high cortisol levels",
    "addison's disease":                 "low cortisol levels",
    "hyperparathyroidism":               "overactive parathyroid glands",
    "hypoparathyroidism":                "underactive parathyroid glands",
    "diabetes insipidus":                "water balance disorder",
    "metabolic syndrome":                "cluster of metabolic disorders",
    "obesity":                           "excess body fat",
    "hyperglycemia":                     "high blood sugar",
    "hypoglycemia":                      "low blood sugar",
    "diabetic ketoacidosis":             "dangerous diabetes complication",
    "hyperosmolar hyperglycemic state":  "severe diabetes complication",
    "gestational diabetes":              "pregnancy-related diabetes",
    "acromegaly":                        "excess growth hormone",
    "gigantism":                         "childhood growth hormone excess",
    "dwarfism":                          "growth hormone deficiency",
    "pheochromocytoma":                  "adrenal gland tumor",
    "carcinoid syndrome":                "hormone-secreting tumor",
    "multiple endocrine neoplasia":      "inherited endocrine tumors",
    "thyroiditis":                       "thyroid inflammation",
    "thyroid cancer":                    "thyroid malignancy",
    "insulinoma":                        "insulin-producing tumor",
    "glucagonoma":                       "glucagon-producing tumor",
    "polycystic ovary syndrome":         "hormonal disorder in women",

    # Hematological (30)
    "anemia":                            "low red blood cells",
    "sickle cell disease":               "abnormal red cell shape",
    "leukemia":                          "blood cancer",
    "lymphoma":                          "lymphatic cancer",
    "thrombocytopenia":                  "low platelet count",
    "thrombocytosis":                    "high platelet count",
    "pancytopenia":                      "low blood cell counts",
    "hemophilia":                        "clotting factor deficiency",
    "disseminated intravascular coagulation": "widespread blood clotting",
    "polycythemia":                      "high red blood cell count",
    "iron deficiency anemia":            "low iron causing anemia",
    "vitamin B12 deficiency":            "pernicious anemia cause",
    "folate deficiency":                 "megaloblastic anemia cause",
    "thalassemia":                       "abnormal hemoglobin production",
    "myelodysplastic syndrome":          "bone marrow failure",
    "myeloproliferative disorder":       "excess blood cell production",
    "hemolytic anemia":                  "red blood cell destruction",
    "aplastic anemia":                   "bone marrow doesn't make blood cells",
    "g6pd deficiency":                   "red blood cell enzyme deficiency",
    "von willebrand disease":            "bleeding disorder",
    "essential thrombocythemia":         "high platelet disorder",
    "myelofibrosis":                     "bone marrow scarring",
    "leukocytosis":                      "high white blood cell count",
    "leukopenia":                        "low white blood cell count",
    "neutropenia":                       "low neutrophil count",
    "eosinophilia":                      "high eosinophil count",
    "mononucleosis":                     "viral infection causing fatigue",
    "blood transfusion":                 "transfer of blood products",
    "bone marrow transplant":            "stem cell transplantation",
    "coagulopathy":                      "blood clotting disorder",

    # Infectious (30)
    "sepsis":                            "body-wide infection",
    "bacteremia":                        "bacteria in blood",
    "viremia":                           "virus in blood",
    "fungemia":                          "fungus in blood",
    "cellulitis":                        "skin infection",
    "abscess":                           "pus-filled cavity",
    "tuberculosis":                      "lung infection by bacteria",
    "HIV/AIDS":                          "viral immune system disease",
    "COVID-19":                          "coronavirus infection",
    "influenza":                         "flu",
    "hepatitis A":                       "liver infection type A",
    "hepatitis B":                       "liver infection type B",
    "hepatitis C":                       "liver infection type C",
    "herpes simplex":                    "cold sore virus",
    "varicella zoster":                  "chickenpox/shingles virus",
    "epstein-barr virus":                "mononucleosis cause",
    "cytomegalovirus":                   "CMV infection",
    "malaria":                           "parasitic blood infection",
    "lyme disease":                      "tick-borne bacterial infection",
    "rocky mountain spotted fever":      "tick-borne illness",
    "syphilis":                          "sexually transmitted infection",
    "gonorrhea":                         "bacterial STI",
    "chlamydia":                         "common bacterial STI",
    "human papillomavirus":              "HPV infection",
    "rabies":                            "viral brain infection",
    "tetanus":                           "bacterial toxin infection",
    "diphtheria":                        "bacterial throat infection",
    "pertussis":                         "whooping cough",
    "meningococcal disease":             "bacterial meningitis cause",
    "staphylococcus infection":          "staph infection",

    # Musculoskeletal (30)
    "osteoarthritis":                    "joint wear-and-tear",
    "osteoporosis":                      "bone thinning",
    "osteopenia":                        "lower bone density",
    "myopathy":                          "muscle disease",
    "tendonitis":                        "tendon inflammation",
    "bursitis":                          "joint fluid sac inflammation",
    "sprain":                            "ligament injury",
    "strain":                            "muscle or tendon injury",
    "fracture":                          "broken bone",
    "dislocation":                       "bone out of joint",
    "rheumatoid arthritis":              "autoimmune joint disease",
    "gout":                              "uric acid crystal arthritis",
    "pseudogout":                        "calcium crystal arthritis",
    "ankylosing spondylitis":           "spinal arthritis",
    "lupus":                             "autoimmune connective tissue disease",
    "scleroderma":                       "hardening of skin and connective tissues",
    "fibromyalgia":                      "widespread musculoskeletal pain",
    "carpal tunnel syndrome":            "wrist nerve compression",
    "rotator cuff tear":                 "shoulder tendon injury",
    "herniated disc":                    "spinal disc problem",
    "scoliosis":                         "abnormal spine curvature",
    "kyphosis":                          "hunchback posture",
    "lordosis":                          "swayback posture",
    "osteomyelitis":                     "bone infection",
    "compartment syndrome":              "dangerous muscle pressure",
    "muscular dystrophy":                "inherited muscle weakness",
    "myasthenia gravis":                 "muscle weakness disorder",
    "polymyositis":                      "muscle inflammation",
    "torticollis":                       "twisted neck muscles",
    "plantar fasciitis":                 "heel pain",

    # Dermatological (30)
    "dermatitis":                        "skin inflammation",
    "eczema":                            "itchy skin rash",
    "psoriasis":                         "scaly skin patches",
    "melanoma":                          "skin cancer",
    "basal cell carcinoma":              "common skin cancer",
    "squamous cell carcinoma":           "skin cell cancer",
    "acne":                              "pimple skin condition",
    "rosacea":                           "facial redness",
    "urticaria":                         "hives",
    "cellulitis":                        "skin infection",
    "impetigo":                          "bacterial skin infection",
    "folliculitis":                      "hair follicle infection",
    "carbuncle":                         "cluster of boils",
    "furuncle":                          "boil",
    "tinea":                             "fungal skin infection",
    "candidiasis":                       "yeast infection",
    "scabies":                           "mite infestation",
    "lice":                              "parasitic infestation",
    "vitiligo":                          "loss of skin pigment",
    "alopecia":                          "hair loss",
    "onychomycosis":                     "nail fungus",
    "paronychia":                        "nail infection",
    "lichen planus":                     "inflammatory skin condition",
    "pemphigus":                         "blistering skin disorder",
    "bullous pemphigoid":                "elderly blistering disorder",
    "erythema multiforme":               "target-like skin lesions",
    "stevens-johnson syndrome":          "severe skin reaction",
    "toxic epidermal necrolysis":        "life-threatening skin condition",
    "keratosis pilaris":                 "chicken skin",
    "molluscum contagiosum":             "viral skin infection",

    # Ophthalmological (20)
    "conjunctivitis":                    "pink eye",
    "glaucoma":                          "high eye pressure",
    "cataract":                          "clouded eye lens",
    "macular degeneration":              "central vision loss",
    "retinopathy":                       "retina damage",
    "keratitis":                         "cornea inflammation",
    "uveitis":                           "uvea inflammation",
    "blepharitis":                       "eyelid inflammation",
    "strabismus":                        "crossed eyes",
    "amblyopia":                         "lazy eye",
    "retinal detachment":                "retina separates from eye",
    "diabetic retinopathy":              "diabetes-related eye damage",
    "hypertensive retinopathy":          "high blood pressure eye damage",
    "optic neuritis":                    "optic nerve inflammation",
    "corneal abrasion":                  "scratch on eye surface",
    "hordeolum":                         "stye",
    "chalazion":                         "eyelid cyst",
    "dry eye syndrome":                  "insufficient tear production",
    "blepharospasm":                     "eyelid twitching",
    "color blindness":                   "color vision deficiency",

    # ENT (20)
    "otitis media":                      "middle ear infection",
    "otitis externa":                    "outer ear infection",
    "sinusitis":                         "sinus inflammation",
    "tonsillitis":                       "tonsil inflammation",
    "pharyngitis":                       "throat inflammation",
    "laryngitis":                        "voice box inflammation",
    "epiglottitis":                      "life-threatening throat infection",
    "rhinitis":                          "nose inflammation",
    "deviated septum":                   "crooked nasal structure",
    "nasal polyps":                      "noncancerous nose growths",
    "hearing loss":                      "reduced hearing ability",
    "tinnitus":                          "ringing in ears",
    "vertigo":                           "spinning sensation",
    "meniere's disease":                 "inner ear balance disorder",
    "otosclerosis":                      "abnormal bone growth in ear",
    "cholesteatoma":                     "abnormal ear skin growth",
    "vocal cord nodules":                "voice overuse lesions",
    "vocal cord polyps":                 "voice abuse growths",
    "laryngeal cancer":                  "voice box cancer",
    "sleep apnea":                       "breathing stops during sleep",

    # Oncology & General (30)
    "carcinoma":                         "cancer of body tissue",
    "sarcoma":                           "cancer of connective tissue",
    "malignant":                         "cancerous",
    "benign":                            "non-cancerous",
    "metastasis":                        "cancer spread",
    "chemotherapy":                      "cancer drug treatment",
    "radiotherapy":                      "cancer radiation treatment",
    "immunotherapy":                     "immune system treatment",
    "biopsy":                            "tissue sample test",
    "CT scan":                           "detailed X-ray scan",
    "MRI":                               "magnetic imaging scan",
    "ultrasound":                        "sound wave imaging",
    "endoscopy":                         "internal camera exam",
    "colonoscopy":                       "colon camera exam",
    "laparoscopy":                       "abdominal camera surgery",
    "mammogram":                         "breast X-ray",
    "pap smear":                         "cervical cancer screening",
    "prostate-specific antigen":         "prostate cancer marker",
    "tumor marker":                      "substance indicating cancer",
    "remission":                         "cancer symptom disappearance",
    "palliative care":                   "symptom relief treatment",
    "hospice":                           "end-of-life care",
    "prognosis":                         "disease outcome prediction",
    "metastasize":                       "cancer spreading",
    "neoplasm":                          "abnormal tissue growth",
    "oncology":                          "cancer medicine",
    "hematology":                        "blood disorder medicine",
    "pathology":                         "disease study",
    "radiology":                         "imaging specialty",
    "histology":                         "tissue study",

    # Additional (50)
    "electroencephalogram":              "brain wave test",
    "lumbar puncture":                   "spinal fluid test",
    "arthroscopy":                       "joint camera exam",
    "pacemaker":                         "heart rate device",
    "defibrillator":                     "heart shock device",
    "ventilator":                        "breathing machine",
    "transfusion":                       "blood transfer",
    "vaccination":                       "disease prevention shot",
    "antibiotic":                        "infection-fighting medicine",
    "antiviral":                         "virus-fighting medicine",
    "analgesic":                         "pain reliever",
    "anticoagulant":                     "blood thinner",
    "stent":                             "small tube to keep the artery open",
    "biopsy":                            "tissue sampling",
    "suture":                            "stitches",
    "catheter":                          "tube for fluid drainage",
    "intravenous":                       "IV therapy",
    "anesthesia":                        "numbing medication",
    "sedation":                          "calming medication",
    "resuscitation":                     "life-saving measures",
    "CPR":                               "cardiopulmonary resuscitation",
    "intensive care unit":               "ICU",
    "emergency room":                    "ER",
    "operating room":                    "OR",
    "rehabilitation":                    "recovery therapy",
    "physical therapy":                  "movement rehabilitation",
    "occupational therapy":              "daily activity rehabilitation",
    "speech therapy":                    "communication rehabilitation",
    "prosthesis":                        "artificial body part",
    "orthosis":                          "supportive device",
    "wheelchair":                        "mobility chair",
    "walker":                            "walking aid",
    "crutches":                          "arm-supported walking aids",
    "cane":                              "walking stick",
    "hospital bed":                      "adjustable patient bed",
    "oxygen mask":                       "oxygen delivery device",
    "nebulizer":                         "breathing treatment device",
    "inhaler":                           "asthma medication device",
    "syringe":                           "injection device",
    "scalpel":                           "surgical knife",
    "forceps":                           "grasping tool",
    "speculum":                          "body opening examination tool",
    "otoscope":                          "ear examination tool",
    "ophthalmoscope":                    "eye examination tool",
    "stethoscope":                       "heart/lung sound tool",
    "blood pressure cuff":               "BP measurement device",
    "thermometer":                       "temperature measurement device",
    "pulse oximeter":                    "oxygen level measurement device",
    "defibrillator":                     "heart rhythm correction device"
}

def apply_lay_terms(text: str) -> str:
    for term in sorted(MEDICAL_DICTIONARY, key=len, reverse=True):
        pattern = re.compile(rf"\b{re.escape(term)}\b", flags=re.IGNORECASE)
        text = pattern.sub(MEDICAL_DICTIONARY[term], text)
    return text

In [20]:
# %%
# 6. Readability Functions

def calculate_readability_scores(text: str) -> dict:
    return {
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(text),
        'smog_index': textstat.smog_index(text),
        'coleman_liau_index': textstat.coleman_liau_index(text),
        'automated_readability_index': textstat.automated_readability_index(text),
        'dale_chall_readability_score': textstat.dale_chall_readability_score(text),
        'difficult_words': textstat.difficult_words(text),
        'linsear_write_formula': textstat.linsear_write_formula(text),
        'gunning_fog': textstat.gunning_fog(text),
        'text_standard': textstat.text_standard(text)
    }


def create_readability_radar_chart(orig: dict, simp: dict):
    metrics = ['flesch_reading_ease','flesch_kincaid_grade','smog_index',
               'coleman_liau_index','automated_readability_index','gunning_fog']
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(r=[orig[m] for m in metrics],
                                  theta=metrics, fill='toself', name='Original'))
    fig.add_trace(go.Scatterpolar(r=[simp[m] for m in metrics],
                                  theta=metrics, fill='toself', name='Simplified'))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True)),
                      showlegend=True,
                      title="Readability Comparison")
    return fig

# Utility to print readability metrics for a result dict
def print_user_readability(res: dict):
    """
    Print only numeric readability metrics for original and simplified texts.
    """
    print("READABILITY METRICS (ORIGINAL):")
    for k, v in res['original_scores'].items():
        if isinstance(v, (int, float)):
            print(f"- {k}: {v:.2f}")
    print("READABILITY METRICS (SIMPLIFIED):")
    for k, v in res['simplified_scores'].items():
        if isinstance(v, (int, float)):
            print(f"- {k}: {v:.2f}")

In [21]:
def identify_medical_terms(text: str) -> list:
    patterns = [
        # 1. Disease and condition suffixes
        r'\b[a-zA-Z]+(?:itis|osis|oma|emia|pathy|plasia|lithiasis|rrhea|uria|phobia|mania)\b',
        
        # 2. Procedure suffixes
        r'\b[a-zA-Z]+(?:graphy|scopy|tomy|ectomy|otomy|plasty|centesis|desis|rrhaphy|stomy|pexy)\b',
        
        # 3. Adjective + noun (descriptive terms)
        r'\b(?:acute|chronic|benign|malignant|idiopathic|hereditary|congenital|infectious|viral|bacterial)\s+[a-z]+\b',
        
        # 4. Multi-word named conditions (e.g., Alzheimer's disease, Parkinson's syndrome)
        r'\b[A-Z][a-z]+(?:\'s)?(?:\s+[A-Z][a-z]+)*\s+(disease|syndrome|disorder|condition)\b',
        
        # 5. Technical diagnostics & imaging (e.g., CT scan, MRI, endoscopy)
        r'\b(?:CT|MRI|PET|X-ray|ultrasound|endoscopy|colonoscopy|biopsy|echocardiogram|angiogram|laparoscopy)\b',
        
        # 6. Common abbreviations or acronyms (e.g., COPD, HIV, ICU)
        r'\b[A-Z]{2,6}\b',
        
        # 7. General multi-word noun phrases (e.g., "heart failure", "lung cancer", "kidney stones")
        r'\b[a-z]+\s+(cancer|failure|infection|disease|disorder|tumor|syndrome|stones|attack|infarction|embolism|occlusion)\b',
        
        # 8. Medical body system pairings (e.g., "renal artery", "pulmonary edema")
        r'\b(?:renal|hepatic|pulmonary|cardiac|neurological|cerebral|gastric|hepatic|endocrine|musculoskeletal|ophthalmic|dermatological|hematologic)\s+[a-z]+\b',
    ]
    
    terms = set()
    for pat in patterns:
        for m in re.finditer(pat, text, re.IGNORECASE):
            terms.add(m.group(0))
    return list(terms)



def generate_term_definitions(terms: list) -> dict:
    defs = {}
    for t in terms:
        key = t.lower()
        if key in MEDICAL_DICTIONARY:
            defs[t] = MEDICAL_DICTIONARY[key]
        else:
            umls_def = get_umls_definition(t)
            defs[t] = umls_def if umls_def else f"Medical term related to '{t.split()[-1]}'"
    return defs

In [22]:
# %%
# 8. Simplification Pipeline

def simplify_medical_text(input_text: str,
                           tokenizer,
                           model) -> dict:
    original = re.sub(r'\s+', ' ', input_text).strip()
    pre_model = apply_lay_terms(original)

    orig_scores = calculate_readability_scores(original)
    pre_scores  = calculate_readability_scores(pre_model)

    toks = tokenizer(pre_model, return_tensors="pt",
                     max_length=MAX_LENGTH, truncation=True)
    with torch.no_grad():
        out = model.generate(
            **toks,
            max_length=MAX_LENGTH,
            min_length=MIN_LENGTH,
            num_beams=4,
            no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,
            early_stopping=True,
            output_attentions=True,
            return_dict_in_generate=True
        )
    simplified = tokenizer.decode(out.sequences[0], skip_special_tokens=True) if hasattr(out, 'sequences') else tokenizer.decode(out[0], skip_special_tokens=True)
    simp_scores = calculate_readability_scores(simplified)

    fired = {t: MEDICAL_DICTIONARY[t]
             for t in MEDICAL_DICTIONARY
             if re.search(rf"\b{re.escape(t)}\b", original, flags=re.IGNORECASE)}
    terms = identify_medical_terms(original)
    umls_terms = {t: defs for t, defs in generate_term_definitions(terms).items() if t.lower() not in MEDICAL_DICTIONARY}

    # capture attention from first layer/head
    att_mat = out.encoder_attentions[0][0,0].cpu().numpy() if hasattr(out, 'encoder_attentions') else None

    return {
        'original_text':      original,
        'pre_model_text':     pre_model,
        'simplified_text':    simplified,
        'original_scores':    orig_scores,
        'premodel_scores':    pre_scores,
        'simplified_scores':  simp_scores,
        'lay_terms_applied':  fired,
        'umls_terms_applied': umls_terms,
        'attention_matrix':   att_mat
    }

In [23]:
# %%
# 9. Quick Examples

tests = [
    "The patient presents with an acute myocardial infarction and ST segment elevation in leads II, III, and aVF.",
    "He has chronic obstructive pulmonary disease and respiratory acidosis on arterial blood gas analysis."
]
for text in tests:
    res = simplify_medical_text(text, tokenizer, model)
    print("ORIGINAL   :", res['original_text'])
    print("PRE‑MODEL  :", res['pre_model_text'])
    print("SIMPLIFIED :", res['simplified_text'])
    print("LAY TERMS  :", res['lay_terms_applied'])
    print("UMLS TERMS :", res['umls_terms_applied'])
    print("-"*60)

ORIGINAL   : The patient presents with an acute myocardial infarction and ST segment elevation in leads II, III, and aVF.
PRE‑MODEL  : The patient presents with an acute heart attack and ST segment elevation in leads II, III, and aVF.
SIMPLIFIED : The patient presents with an acute heart attack and ST segment elevation in leads II, III, and aVF . SEP> The patient has a heart attack , a stroke , and an affection .
LAY TERMS  : {'myocardial infarction': 'heart attack'}
UMLS TERMS : {'with': "Medical term related to 'with'", 'ST': "Medical term related to 'ST'", 'leads': "Medical term related to 'leads'", 'aVF': "Medical term related to 'aVF'", 'an': "Medical term related to 'an'", 'acute': "Medical term related to 'acute'", 'II': "Medical term related to 'II'", 'III': "Medical term related to 'III'", 'The': "Medical term related to 'The'", 'in': "Medical term related to 'in'", 'and': "Medical term related to 'and'", 'acute myocardial': "Medical term related to 'myocardial'"}
------------

In [24]:
# %%
# 10. Readability & Attention Visualization for First Example
example_text = tests[0]
res = simplify_medical_text(example_text, tokenizer, model)

# Radar Chart
fig = create_readability_radar_chart(res['original_scores'], res['simplified_scores'])
fig.show()

# Attention Heatmap (first 20 tokens)
if res['attention_matrix'] is not None:
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(res['pre_model_text'], truncation=True, max_length=MAX_LENGTH))
    heat = px.imshow(
        res['attention_matrix'][:20, :20],
        x=tokens[:20], y=tokens[:20],
        labels=dict(color="Attention"),
        title="Attention (layer 0/head 0)"
    )
    heat.show()


In [25]:
# %%
# 11. User Input Example & Visualization
user_text = """
The patient is a 60-year-old male with a history of coronary artery disease who presented with acute onset substernal chest pain.
Electrocardiogram showed ST-segment elevation in leads II, III, and aVF.
"""
user_res = simplify_medical_text(user_text, tokenizer, model)

print("Original:   ", user_res['original_text'])
#print("Pre‑model:  ", user_res['pre_model_text'])
print("Simplified: ", user_res['simplified_text'])
print("Lay Terms:  ", user_res['lay_terms_applied'])
print("UMLS Terms: ", user_res['umls_terms_applied'])

# Radar
fig2 = create_readability_radar_chart(user_res['original_scores'], user_res['simplified_scores'])
fig2.show()

# Attention
tokens2 = tokenizer.convert_ids_to_tokens(tokenizer.encode(user_res['pre_model_text'], truncation=True, max_length=MAX_LENGTH))
if user_res['attention_matrix'] is not None:
    heat2 = px.imshow(
        user_res['attention_matrix'][:20, :20],
        x=tokens2[:20], y=tokens2[:20],
        labels=dict(color="Attention"),
        title="User Text Attention (layer 0/head 0)"
    )
    heat2.show()
# Print readability scores for user input example alone
print_user_readability(user_res)

Original:    The patient is a 60-year-old male with a history of coronary artery disease who presented with acute onset substernal chest pain. Electrocardiogram showed ST-segment elevation in leads II, III, and aVF.
Simplified:  The patient is a 60-year-old male with a history of heart artery blockage who presented with acute onset substernal chest pain . SEP> Electrocardiogram showed ST-segment elevation in leads II, III, and aVF .
Lay Terms:   {'coronary artery disease': 'heart artery blockage'}
UMLS Terms:  {'ST': "Medical term related to 'ST'", 'artery': "Medical term related to 'artery'", 'onset': "Medical term related to 'onset'", 'acute': "Medical term related to 'acute'", 'history of coronary artery disease': "Medical term related to 'disease'", 'The': "Medical term related to 'The'", 'who': "Medical term related to 'who'", 'with': "Medical term related to 'with'", 'year': "Medical term related to 'year'", 'in': "Medical term related to 'in'", 'is': "Medical term related to 'is

READABILITY METRICS (ORIGINAL):
- flesch_reading_ease: 38.82
- flesch_kincaid_grade: 11.70
- smog_index: 0.00
- coleman_liau_index: 13.28
- automated_readability_index: 12.50
- dale_chall_readability_score: 13.57
- difficult_words: 11.00
- linsear_write_formula: 11.25
- gunning_fog: 13.94
READABILITY METRICS (SIMPLIFIED):
- flesch_reading_ease: 46.78
- flesch_kincaid_grade: 10.70
- smog_index: 0.00
- coleman_liau_index: 12.53
- automated_readability_index: 12.20
- dale_chall_readability_score: 13.31
- difficult_words: 10.00
- linsear_write_formula: 11.50
- gunning_fog: 12.65


In [29]:
# %%
# 12. Interactive Simplification via input()
user_text = input("Enter medical text to simplify:")
res = simplify_medical_text(user_text, tokenizer, model)

print("--- Simplification Result ---")
print("Original Text: ", res['original_text'])
print("Simplified Text: ", res['simplified_text'])
print_user_readability(res)

# Optionally visualize
fig3 = create_readability_radar_chart(res['original_scores'], res['simplified_scores'])
fig3.show()
if res['attention_matrix'] is not None:
    tokens3 = tokenizer.convert_ids_to_tokens(tokenizer.encode(res['pre_model_text'], truncation=True, max_length=MAX_LENGTH))
    heat3 = px.imshow(
        res['attention_matrix'][:20, :20],
        x=tokens3[:20], y=tokens3[:20],
        labels=dict(color="Attention"),
        title="Interactive Text Attention (layer 0/head 0)"
    )
    heat3.show()

Enter medical text to simplify: The patient was diagnosed with myocardial infarction and underwent cardiac catheterization due to severe coronary artery disease and arrhythmia.


--- Simplification Result ---
Original Text:  The patient was diagnosed with myocardial infarction and underwent cardiac catheterization due to severe coronary artery disease and arrhythmia.
Simplified Text:  The patient was diagnosed with heart attack and underwent heart tube examination due to severe heart artery blockage and abnormal heartbeat . SEP> The patient had a heart attack , and had an abnormal heart rate .
READABILITY METRICS (ORIGINAL):
- flesch_reading_ease: -15.50
- flesch_kincaid_grade: 20.10
- smog_index: 0.00
- coleman_liau_index: 20.88
- automated_readability_index: 19.30
- dale_chall_readability_score: 14.55
- difficult_words: 12.00
- linsear_write_formula: 18.50
- gunning_fog: 26.55
READABILITY METRICS (SIMPLIFIED):
- flesch_reading_ease: 54.22
- flesch_kincaid_grade: 9.90
- smog_index: 0.00
- coleman_liau_index: 11.77
- automated_readability_index: 11.50
- dale_chall_readability_score: 9.59
- difficult_words: 9.00
- linsear_write_formula: 12.25
- gunning_fog: 12.6

# 

# Other Pretained medical models

In [1]:
# Install the Hugging Face Hub helper
!pip install huggingface_hub


In [ ]:
# login_hf.py
from huggingface_hub import login

# Replace "YOUR_TOKEN_HERE" with your actual token string
login(token="")

print("✅ Logged in to Hugging Face!")


✅ Logged in to Hugging Face!


In [3]:
# 13. Install and setup Bio-Medical-Llama-3-8B
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [6]:
# %% 
import torch
import textstat
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import plotly.express as px
import plotly.graph_objects as go

# Readability utilities
def calculate_readability_scores(text: str) -> dict:
    return {
        'flesch_reading_ease': textstat.flesch_reading_ease(text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(text),
        'smog_index': textstat.smog_index(text),
        'coleman_liau_index': textstat.coleman_liau_index(text),
        'automated_readability_index': textstat.automated_readability_index(text),
        'gunning_fog': textstat.gunning_fog(text)
    }

def create_readability_radar_chart(orig: dict, simp: dict):
    metrics = ['flesch_reading_ease','flesch_kincaid_grade','smog_index',
               'coleman_liau_index','automated_readability_index','gunning_fog']
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=[orig[m] for m in metrics],
        theta=metrics,
        fill='toself',
        name='Original'
    ))
    fig.add_trace(go.Scatterpolar(
        r=[simp[m] for m in metrics],
        theta=metrics,
        fill='toself',
        name='Simplified'
    ))
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True)),
        showlegend=True,
        title="Readability Comparison"
    )
    return fig

# ---- load your chosen model ----
# MODEL_NAME = "Falconsai/medical_summarization"
# MODEL_NAME = "GanjinZero/biobart-v2-base"
MODEL_NAME = "jkhan447/t5-medical-text-simplification"
print(f"Loading model {MODEL_NAME}…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Simple wrapper
def simplify(text: str) -> str:
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs,
                             num_beams=4,
                             max_length=512,
                             early_stopping=True,
                             output_attentions=True,
                             return_dict_in_generate=True)
    return tokenizer.decode(outputs.sequences[0], skip_special_tokens=True), outputs

# ---- example ----
example = "The patient presents with acute myocardial infarction and hypertension."
simplified, gen_outputs = simplify(example)

print("Original:  ", example)
print("Simplified:", simplified)

# ---- readability ----
orig_scores = calculate_readability_scores(example)
simp_scores = calculate_readability_scores(simplified)

print("\nReadability Metrics:")
for m in orig_scores:
    print(f"- {m}: {orig_scores[m]:.2f} → {simp_scores[m]:.2f}")

# ---- radar chart ----
fig = create_readability_radar_chart(orig_scores, simp_scores)
fig.show()

# ---- attention heatmap ----
# extract encoder attentions (first layer & first head)
att = gen_outputs.encoder_attentions[0][0,0].cpu().numpy()
tokens = tokenizer.convert_ids_to_tokens(
    tokenizer(example, return_tensors="pt", truncation=True, max_length=512).input_ids[0]
)

heat = px.imshow(
    att[:20, :20],
    x=[t.replace("Ġ","") for t in tokens[:20]],
    y=[t.replace("Ġ","") for t in tokens[:20]],
    labels=dict(x="Input Token", y="Input Token", color="Attention"),
    title="Attention Heatmap (Layer 0, Head 0)"
)
heat.show()


Loading model jkhan447/t5-medical-text-simplification…


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

...

2025-04-24 04:44:22.963150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745469863.152374      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745469863.205625      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Original:   The patient presents with acute myocardial infarction and hypertension.
Simplified: The patient has acute myocardial infarction and hypertension.

Readability Metrics:
- flesch_reading_ease: 3.12 → -4.33
- flesch_kincaid_grade: 15.10 → 15.90
- smog_index: 0.00 → 0.00
- coleman_liau_index: 20.91 → 18.81
- automated_readability_index: 16.00 → 14.40
- gunning_fog: 16.93 → 18.20
